In [26]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D #For 3D Plots

#Required by Jupyter for inlining images
%matplotlib inline

Function:

In [28]:
def match_datasets(df_1, col_index_1, df_2, col_index_2, lower_epsilon, upper_epsilon):
    #col_indexes are the names of the columns to search (eg. 'Time Stamp')
    #Epsilon is the tolerance in seconds
    
    lenDf = len(df_1.columns) + len(df_2.columns)
    df = pd.DataFrame()#columns=list(range(0,lenDf)))
    
    matched = 0 #Number of elements matched
    dropped_df_1 = 0 #Number of elements dropped from df_1
    dropped_df_2 = 0 #Number of elements dropped from df_2
    
    #Ensure Dataframes are sorted by timestamp
    df_1 = df_1.sort_values(by=[col_index_1]).reset_index()
    df_2 = df_2.sort_values(by=[col_index_2]).reset_index()
    
    i = 0 #keeps track of index of df_1
    j = 0 #keeps track of index of df_2
    while i < len(df_1[col_index_1])-1 and j < len(df_2[col_index_2])-1:
        #if abs(df_1[col_index_1][i]-df_2[col_index_2][j]) < epsilon:
        if (df_1[col_index_1][i]-df_2[col_index_2][j]) > lower_epsilon and (df_1[col_index_1][i]-df_2[col_index_2][j]) < upper_epsilon:
            #Case 1:
            #Points matched
            matched = matched + 1
            #Add to new DataFrame:
            row = pd.Series(df_1.iloc[i,:].append(df_2.iloc[j,:])).to_frame().T
            df = df.append(row,ignore_index=True)
            #Increment both i and j
            i = i + 1
            j = j + 1
        #Lookahead on i
        elif df_1[col_index_1][i] < df_2[col_index_2][j]:
            #Case 2:
            i = i + 1
            dropped_df_1 = dropped_df_1 + 1 
            while df_1[col_index_1][i] < df_2[col_index_2][j]:
                #if abs(df_1[col_index_1][i]-df_2[col_index_2][j]) < epsilon:
                if (df_1[col_index_1][i]-df_2[col_index_2][j]) > lower_epsilon and (df_1[col_index_1][i]-df_2[col_index_2][j]) < upper_epsilon:
                    #Points Matched
                    matched = matched + 1
                    i = i + 1
                    j = j + 1
                    break
                else:
                    i = i + 1
                    dropped_df_1 = dropped_df_1 + 1
        elif df_1[col_index_1][i] > df_2[col_index_2][j]:
            #Case 3:
            j = j + 1
            dropped_df_2 = dropped_df_2 + 1 
            while df_1[col_index_1][i] > df_2[col_index_2][j]:
                #if abs(df_1[col_index_1][i]-df_2[col_index_2][j]) < epsilon:
                if (df_1[col_index_1][i]-df_2[col_index_2][j]) > lower_epsilon and (df_1[col_index_1][i]-df_2[col_index_2][j]) < upper_epsilon:
                    #Points Matched
                    matched = matched + 1
                    i = i + 1
                    j = j + 1
                    break
                else:
                    j = j + 1
                    dropped_df_2 = dropped_df_2 + 1
                
        
 
    print('Matched: ' + str(matched))
    print('Dropped from 1st df: ' + str(dropped_df_1))
    print('Dropped from 2nd df: ' + str(dropped_df_2))
    #return pd.DataFrame.from_records(rows)
    #print(rows)  
    
    return df

# Beam 1:

### TL_BPM:

In [29]:
TL_BPM = pd.read_pickle('../Data/Beam1_TL_BPM.pkl')

In [30]:
TL_BPM.head()

,Time Stamp,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,MSE
0,1.534476e+09,-0.114,-0.035,-1.063,-0.024,-1.353,-1.518,-1.321,-1.025,0.254,-0.879,0.341,0.049,-1.138,-1.413,0.088,-1.494,-0.411,-0.820,0.000000
1,1.534476e+09,-0.062,-0.134,-0.786,-0.088,-1.211,-1.638,-1.044,-0.986,0.294,-0.837,0.301,0.101,-1.127,-1.339,0.204,-1.436,-0.358,-0.765,0.013448
2,1.534476e+09,-0.146,0.082,-1.066,0.090,-1.405,-1.392,-1.333,-0.891,0.286,-0.774,0.346,-0.035,-1.098,-1.444,0.073,-1.414,-0.433,-0.730,0.005628
3,1.534476e+09,-0.104,-0.037,-0.872,-0.020,-1.259,-1.500,-1.106,-0.912,0.269,-0.762,0.311,0.023,-1.143,-1.347,0.132,-1.396,-0.379,-0.668,0.008905
4,1.534477e+09,-0.014,-0.008,-0.859,0.053,-1.343,-1.365,-1.211,-0.913,0.224,-0.740,0.250,0.025,-0.971,-1.447,0.268,-1.489,-0.406,-0.696,0.011799


In [31]:
Data = pd.DataFrame(data={'Time Stamp BPM':TL_BPM['Time Stamp']})

In [32]:
## Renaming Columns and adding them to the Dataset
j = 1
for i in range(1,19):
    column = TL_BPM.iloc[:,i]
    Data['BPM Reading '+str(j)] = column
    j = j+1
Data.head()

,Time Stamp BPM,BPM Reading 1,BPM Reading 2,BPM Reading 3,BPM Reading 4,BPM Reading 5,BPM Reading 6,BPM Reading 7,BPM Reading 8,BPM Reading 9,BPM Reading 10,BPM Reading 11,BPM Reading 12,BPM Reading 13,BPM Reading 14,BPM Reading 15,BPM Reading 16,BPM Reading 17,BPM Reading 18
0,1.534476e+09,-0.114,-0.035,-1.063,-0.024,-1.353,-1.518,-1.321,-1.025,0.254,-0.879,0.341,0.049,-1.138,-1.413,0.088,-1.494,-0.411,-0.820
1,1.534476e+09,-0.062,-0.134,-0.786,-0.088,-1.211,-1.638,-1.044,-0.986,0.294,-0.837,0.301,0.101,-1.127,-1.339,0.204,-1.436,-0.358,-0.765
2,1.534476e+09,-0.146,0.082,-1.066,0.090,-1.405,-1.392,-1.333,-0.891,0.286,-0.774,0.346,-0.035,-1.098,-1.444,0.073,-1.414,-0.433,-0.730
3,1.534476e+09,-0.104,-0.037,-0.872,-0.020,-1.259,-1.500,-1.106,-0.912,0.269,-0.762,0.311,0.023,-1.143,-1.347,0.132,-1.396,-0.379,-0.668
4,1.534477e+09,-0.014,-0.008,-0.859,0.053,-1.343,-1.365,-1.211,-0.913,0.224,-0.740,0.250,0.025,-0.971,-1.447,0.268,-1.489,-0.406,-0.696


In [33]:
Data['BPM MSE'] = TL_BPM['MSE']

### TDI_BLM:

In [34]:
BLM = pd.read_pickle('../Data/Beam1_BLM_NN.pkl')

We'll just use Reading 1 since we showed they're highly correlated

In [35]:
BLM = pd.DataFrame(data={'Time Stamp BLM':BLM['Time Stamp'], 'BLM': BLM['Reading 1']})

#### Merge the 2 Datasets:

In [36]:
Data = match_datasets(BLM,'Time Stamp BLM',Data,'Time Stamp BPM', 14 ,20)
Data = Data.drop(['index'],axis=1)

Matched: 1402
Dropped from 1st df: 492
Dropped from 2nd df: 17


### Intensities (LHC-SPS):

In [37]:
Intensities = pd.read_pickle('../Data/intensities.pkl')
Intensities = pd.DataFrame({'Time Stamp SPS': Intensities['Time Stamp']-Intensities['Time Differences'],
                            'Time Stamp LHC': Intensities['Time Stamp'],
                            'LHC-SPS':Intensities['Change in Intensities']})

In [38]:
Data = match_datasets(Intensities,'Time Stamp LHC',Data,'Time Stamp BPM', 14 ,20)
Data = Data.drop(['index'],axis=1)

Matched: 893
Dropped from 1st df: 128
Dropped from 2nd df: 1


### Number of Bunches:

In [39]:
Bunches = pd.read_pickle('../Data/Beam1_Bunches.pkl')
Bunches = Bunches.rename(columns={'Time Stamp':'Time Stamp Bunches'})

In [40]:
Data = match_datasets(Data,'Time Stamp LHC',Bunches,'Time Stamp Bunches', -5 ,5)
Data = Data.drop(['index'],axis=1)

Matched: 886
Dropped from 1st df: 5
Dropped from 2nd df: 37


In [41]:
Data = Data.rename(columns={'Change':'Change in Number of Bunches'})

### Abort Gap:

In [43]:
Abort_Gap = pd.read_pickle('../Data/Beam1_AbortGap.pkl')

In [44]:
Data = match_datasets(Abort_Gap,'Time Stamp Abort Gap',Data,'Time Stamp LHC', -6 ,6)
Data = Data.drop(['index'],axis=1)

Matched: 879
Dropped from 1st df: 315
Dropped from 2nd df: 1


In [45]:
Data.head()

,Time Stamp Abort Gap,Abort Gap Reading,Time Stamp SPS,Time Stamp LHC,LHC-SPS,Time Stamp BLM,BLM,Time Stamp BPM,BPM Reading 1,BPM Reading 2,...,BPM Reading 13,BPM Reading 14,BPM Reading 15,BPM Reading 16,BPM Reading 17,BPM Reading 18,BPM MSE,Time Stamp Bunches,Number of Bunches,Change in Number of Bunches
0,1.534476e+09,5.136406e+08,1.534476e+09,1.534476e+09,-9.910938e+08,1.534476e+09,0.003960,1.534476e+09,-0.062,-0.134,...,-1.127,-1.339,0.204,-1.436,-0.358,-0.765,0.013448,1.534476e+09,109.0,96.0
1,1.534476e+09,1.605127e+08,1.534476e+09,1.534476e+09,1.496010e+09,1.534476e+09,0.002312,1.534476e+09,-0.146,0.082,...,-1.098,-1.444,0.073,-1.414,-0.433,-0.730,0.005628,1.534476e+09,121.0,12.0
2,1.534476e+09,-2.453854e+08,1.534476e+09,1.534476e+09,1.760888e+10,1.534476e+09,0.004158,1.534476e+09,-0.104,-0.037,...,-1.143,-1.347,0.132,-1.396,-0.379,-0.668,0.008905,1.534476e+09,265.0,144.0
3,1.534477e+09,3.852305e+08,1.534477e+09,1.534477e+09,-1.270084e+09,1.534477e+09,0.003327,1.534477e+09,-0.014,-0.008,...,-0.971,-1.447,0.268,-1.489,-0.406,-0.696,0.011799,1.534477e+09,277.0,12.0
4,1.534477e+09,-2.889211e+08,1.534477e+09,1.534477e+09,1.024349e+10,1.534477e+09,0.004684,1.534477e+09,0.008,-0.017,...,-1.130,-1.332,0.253,-1.402,-0.345,-0.696,0.021249,1.534477e+09,421.0,144.0


### Save to Pickle 

In [46]:
Data.to_pickle('../Data/Beam1_MergedRawData.pkl')